<a href="https://colab.research.google.com/github/BS-Varun/Shakespeare/blob/main/Shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.3 MB/s eta 0:00:00


In [2]:
!pip install requests
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.0 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset

dataset = load_dataset("tiny_shakespeare")




Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

Dataset tiny_shakespeare downloaded and prepared to /root/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/b5b13969f09fe8707337f6cb296314fbe06960bd9a868dca39e713e163d27b5e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.1 MB/s eta 0:00:00


In [23]:
from datasets import load_dataset
from transformers import GPT2Tokenizer

# Load the Tiny Shakespeare dataset
dataset = load_dataset("tiny_shakespeare")

# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Assign a special token as the padding token
# Add a padding token to the tokenizer

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Define a function to preprocess the dataset
def preprocess_dataset(example):
    # Tokenize the text
    encoded_inputs = tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

    # Update the example with tokenized inputs
    example["input_ids"] = encoded_inputs["input_ids"]
    example["attention_mask"] = encoded_inputs["attention_mask"]
    # Add 'labels' key with target values
    example["labels"] = encoded_inputs["input_ids"]

    return example

# Preprocess the dataset
preprocessed_dataset = dataset.map(preprocess_dataset, batched=True)

# Split the dataset into train and test sets
train_dataset = preprocessed_dataset["train"]
test_dataset = preprocessed_dataset["test"]




  0%|          | 0/3 [00:00<?, ?it/s]

Using pad_token, but it is not set yet.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [7]:
from transformers import GPT2LMHeadModel, GPT2Config, GPT2Tokenizer, Trainer, TrainingArguments


In [8]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW

# Load the pre-trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

# Define the custom loss function
loss_fn = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Define the training loop
def train(model, optimizer, data_loader):
    model.train()
    total_loss = 0

    for batch in data_loader:
        # Convert batch items to tensors
        input_ids = torch.tensor(batch["input_ids"]).to(device)
        attention_mask = torch.tensor(batch["attention_mask"]).to(device)
        labels = torch.tensor(batch["input_ids"]).to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(data_loader)

# Define the number of training epochs
num_epochs = 5

# Define the data loader (adjust batch size and other parameters as needed)
data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)

# Fine-tuning loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    avg_loss = train(model, optimizer, data_loader)
    print(f"Average Loss: {avg_loss}")

# Save the fine-tuned model
model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5
Average Loss: 4.179553031921387
Epoch 2/5
Average Loss: 4.082951545715332
Epoch 3/5
Average Loss: 3.9381186962127686
Epoch 4/5
Average Loss: 4.156772613525391
Epoch 5/5
Average Loss: 3.8687500953674316


('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/vocab.json',
 'fine_tuned_model/merges.txt',
 'fine_tuned_model/added_tokens.json')

In [11]:
!pip install --upgrade transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import requests
from bs4 import BeautifulSoup
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned model and tokenizer
model_path = 'fine_tuned_model'
tokenizer_path = 'fine_tuned_model'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path,padding_side='left')
model.to(device)
model.eval()
tokenizer.pad_token = tokenizer.eos_token 
# Function to scrape the web and extract relevant information
def scrape_urls(query):
    search_url = f'https://www.google.com/search?q={query}&num=5'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    try:
        response = requests.get(search_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        search_results = soup.select('.kCrYT > a')
        return [result['href'] for result in search_results]
    except requests.exceptions.RequestException:
        return None

# Function to generate a response using the fine-tuned model
def generate_response(question, model):
    inputs = tokenizer.encode_plus(question, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Generate a response
    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=1000)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Function to search the web, extract relevant information, and generate a response
def search_and_respond(question, model):
    search_results = scrape_urls(question)
    if search_results:
        # Extract the relevant information from the search results
        # and generate a response using the fine-tuned model
        response = generate_response(question, model)
        return response
    else:
        return "No search results found for the question."

# Example usage
question = "What is India?"
response = search_and_respond(question, model)
print(response)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is India?

India is a country of about 1.5 billion people. It is the second largest economy in the world after China. It is the second largest economy in the world after the United States. It is the second largest economy in the world after the United Kingdom. It is the second largest economy in the world after the United States. It is the second largest economy in the world after the United States. It is the second largest economy in the world after the United States. It is the second largest economy in the world after the United States. It is the second largest economy in the world after the United States. It is the second largest economy in the world after the United States. It is the second largest economy in the world after the United States. It is the second largest economy in the world after the United States. It is the second largest economy in the world after the United States. It is the second largest economy in the world after the United States. It is the second largest

In [15]:
!pip install spacy
!pip install sympy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Shakespeare as **Mathematician**

In [18]:
import sympy as sp
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned GPT-2 model and tokenizer
model_path = "fine_tuned_model"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

def perform_math_calculation(question):
    # Remove any leading/trailing whitespaces
    question = question.strip()

    # Check if the question contains "factorial" keyword
    if "factorial" in question:
        # Extract the number from the question
        number = int(''.join(filter(str.isdigit, question)))

        # Calculate the factorial using sympy
        result = sp.factorial(number)

        response = f"The factorial of {number} is {result}"
    else:
        response = "I'm sorry, I couldn't understand the mathematical question."

    return response

def generate_response(question):
    # Tokenize the input question
    input_ids = tokenizer.encode(question, return_tensors="pt")

    # Generate the response using the fine-tuned model
    output = model.generate(input_ids=input_ids, max_length=100)

    # Decode the response and convert it to a string
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    return response

def answer_question(question):
    # Check if the question requires a mathematical calculation
    if any(keyword in question for keyword in ["factorial"]):
        response = perform_math_calculation(question)
    else:
        response = generate_response(question)

    return response

# Example usage
question = "What is the factorial of 10?"
response = answer_question(question)
print(response)


The factorial of 10 is 3628800


In [22]:
import sympy as sp
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned GPT-2 model and tokenizer
model_path = "fine_tuned_model"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

def perform_math_calculation(question):
    # Remove any leading/trailing whitespaces
    question = question.strip()

    # Check if the question contains any mathematical keywords
    if "factorial" in question:
        # Extract the number from the question
        number = int(''.join(filter(str.isdigit, question)))

        # Calculate the factorial using sympy
        result = sp.factorial(number)

        response = f"The factorial of {number} is {result}"
    elif "square root" in question:
        # Extract the number from the question
        number = int(''.join(filter(str.isdigit, question)))

        # Calculate the square root using sympy
        result = sp.sqrt(number)

        response = f"The square root of {number} is {result}"
    elif "power of" in question:
        # Extract the base and exponent from the question
        base, exponent = question.split("power of")[1].strip().split()

        # Evaluate the power using sympy
        result = sp.Pow(int(base), int(exponent))

        response = f"{base} raised to the power of {exponent} is {result}"
    else:
        response = "I'm sorry, I couldn't understand the mathematical question."

    return response

def generate_response(question):
    # Tokenize the input question
    input_ids = tokenizer.encode(question, return_tensors="pt")

    # Generate the response using the fine-tuned model
    output = model.generate(input_ids=input_ids, max_length=100)

    # Decode the response and convert it to a string
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    return response

def answer_question(question):
    # Check if the question requires a mathematical calculation
    if any(keyword in question for keyword in ["factorial", "square root", "power of"]):
        response = perform_math_calculation(question)
    else:
        response = generate_response(question)

    return response

# Example usage
question = "What is the square root of 16?"
response = answer_question(question)
print(response)


The square root of 16 is 4
